In [1]:
import bs4
import ollama
import chromadb
from langchain import hub
from operator import itemgetter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Documents for RAG system
documents = [
    Document(page_content="RAG combines information retrieval with LLMs to generate more accurate and context-aware responses. In RAG, relevant documents are retrieved and used as input to improve the LLM's performance.RAG enhances the accuracy by providing real-time access to up-to-date information."),
    Document(page_content="LLMs are AI models trained on vast datasets to understand and generate human-like text. LLMs excel at tasks like language translation, summarization, and question answering. With vast knowledge, LLMs can assist in automating content creation and customer support."),
]


In [3]:
# Sentence-based text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=70, chunk_overlap=20)   # chunk_overlap = Overlaps the chunk to help us chain chunck together
splits = text_splitter.split_documents(documents)

# Generate embeddings and create vectorstore
embeddings = OllamaEmbeddings(model="llama3.1")
vectorstore = Chroma.from_documents(splits, embeddings)

# Create a retriever from the vectorstore
retriever = vectorstore.as_retriever()        #Retriver is used to find chunks in the vector store that are similar to the given question.

# The 'prompt' variable is a pre-configured prompt that can be used to generate answers to questions by using retrieved contextual information.
prompt = hub.pull("rlm/rag-prompt")

# Initialize the LLM
llm = ChatOllama(model="llama3.1", temperature=0)

# Chain      

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser() #Parser is used to extract the text answer by using output parser.
)


C:\Users\sathw\anaconda3\lib\site-packages\langsmith\client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
C:\Users\sathw\anaconda3\lib\site-packages\langsmith\client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [4]:
# Define a question and invoke the chain
question = "What are the task of LLM?"
result = chain.invoke({'question': question})

# Print the result
print(result)

LLMs (Large Language Models) can assist in automating content creation, improving performance through input from retrieved documents, and generating accurate responses for tasks such as question answering and customer support. They can also be used to improve the performance of Retrieval-Augmented Generation (RAG) models. Overall, LLMs are designed to facilitate various tasks related to language understanding and generation.


In [5]:
# Define a question and invoke the chain
question = "What family is a llama related to?"
result = chain.invoke({'question': question})

# Print the result
print(result)

I don't know. The provided context does not mention llamas or their family relationships. It appears to be about Large Language Models (LLMs) and information retrieval.
